## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

### Load requirements and data

In [21]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical, layer_utils, plot_model

from keras.layers import (Embedding, Input, Dense, 
                          Activation, Conv1D, MaxPooling1D, Flatten)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard, Callback
import keras.backend as K
from keras.losses import binary_crossentropy

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score 
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.utils import class_weight

import tensorflow as tf


### Envoronmental vars

In [25]:
# env vars are not being read for some reason 💩

DATADIR=os.getenv('DATADIR')
if ~isinstance(DATADIR, str):
    DATADIR='/data'

## Hyperparameters

In [26]:
MAX_SEQUENCE_LENGTH =1000
EMBEDDING_DIM = 100 # used 6B.100d.txt
P_THRESHOLD = 0.4 #Threshold for probability of being assigned to class
POS_RATIO = 0.5 #ratio of positive to negative for each class in weighted binary cross entropy loss function

### Read in data
Content items tagged to level 2 taxons or lower in the topic taxonomy

In [27]:
labelled_level2 = pd.read_csv(
    os.path.join(DATADIR, 'labelled_level2.csv.gz'),
    dtype=object, 
    compression='gzip'
)

### Create target/Y 

Note: when using the categorical_crossentropy loss, your targets should be in categorical format (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros expect for a 1 at the index corresponding to the class of the sample). In order to convert integer targets into categorical targets, you can use the Keras utility to_categorical

In [28]:
#COLLAPSE World level2taxons
labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

In [29]:
labelled_level2['level2taxon'] = labelled_level2['level2taxon'].astype('category')

labels = labelled_level2['level2taxon'].cat.codes 

In [30]:
labels_index = dict(zip((labels), labelled_level2['level2taxon']))

In multilabel learning, the joint set of binary classification tasks is expressed with label binary indicator array: each sample is one row of a 2d array of shape (n_samples, n_classes) with binary values:  
the one, i.e. the non zero elements, corresponds to the subset of labels.  
An array such as np.array([[1, 0, 0], [0, 1, 1], [0, 0, 0]]) represents label 0 in the first sample, labels 1 and 2 in the second sample, and no labels in the third sample.  
Producing multilabel data as a list of sets of labels may be more intuitive. The MultiLabelBinarizer transformer can be used to convert between a collection of collections of labels and the indicator format.

####  first reshape wide to get columns for each level2taxon and row number = number unique urls

In [31]:
#get a smaller copy of data for pivoting ease (think you can work from full data actually and other cols get droopedauto)

level2_reduced = labelled_level2[['content_id', 'level2taxon', 'combined_text']].copy()

#how many level2taxons are there?
print(level2_reduced.level2taxon.nunique())

#count the number of taxons per content item into new column
level2_reduced['num_taxon_per_content'] = level2_reduced.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
level2_reduced['level2taxon_code'] = level2_reduced.level2taxon.astype('category').cat.codes + 1

210


In [32]:
#reshape to wide per taxon and keep the combined text so indexing is consistent when splitting X from Y

multilabel = (level2_reduced.pivot_table(index=['content_id', 'combined_text'], 
                  columns='level2taxon_code', 
                  values='num_taxon_per_content'))
print(level2_reduced.shape)
print(multilabel.shape)

(173560, 5)
(114048, 210)


In [33]:
#convert the number_of_taxons_per_content values to 1, meaning there was an entry for this taxon and this content_id, 0 otherwise
binary_multilabel = multilabel.notnull().astype('int')

In [34]:
#will convert columns to an array of shape
binary_multilabel[list(binary_multilabel.columns)].values.shape

(114048, 210)

In [35]:
#convert columns to an array. Each row represents a content item, each column an individual taxon
binary_multilabel = binary_multilabel[list(binary_multilabel.columns)].values

In [36]:
type(binary_multilabel)

numpy.ndarray

### Create language data/X

format our text samples and labels into tensors that can be fed into a neural network. To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences.

In [37]:
multilabel.index.names
texts = multilabel.index.get_level_values('combined_text')
texts.shape

(114048,)

In [38]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) #yield one sequence per input text

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH

Found 213132 unique tokens.


In [39]:
print('Shape of label tensor:', binary_multilabel.shape)
print('Shape of data tensor:', data.shape)

Shape of label tensor: (114048, 210)
Shape of data tensor: (114048, 1000)


### Data split
- Training data = 80%
- validation data = 10%
- Test data = 10%

In [40]:
# shuffle data and standardise indices
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = binary_multilabel[indices]

In [41]:
nb_test_samples = int(0.1 * data.shape[0]) #validation split
print('nb_test samples:', nb_test_samples)

nb_val_samples = int(0.2 * data.shape[0]) #validation split
print('nb_val samples:', nb_val_samples)

nb_training_samples = int(0.8 * data.shape[0]) #validation split
print('nb_training samples:', nb_training_samples)

nb_test samples: 11404
nb_val samples: 22809
nb_training samples: 91238


In [42]:
x_train = data[:-nb_val_samples]
print('Shape of x_train:', x_train.shape)
y_train = labels[:-nb_val_samples]
print('Shape of y_train:', y_train.shape)

x_val = data[-nb_val_samples:-nb_test_samples]
print('Shape of x_val:', x_val.shape)
y_val = labels[-nb_val_samples:-nb_test_samples]
print('Shape of y_val:', y_val.shape)

x_test = data[-nb_test_samples:]
print('Shape of x_val:', x_val.shape)
y_test = labels[-nb_test_samples:]
print('Shape of y_val:', y_val.shape)

Shape of x_train: (91239, 1000)
Shape of y_train: (91239, 210)
Shape of x_val: (11405, 1000)
Shape of y_val: (11405, 210)
Shape of x_val: (11405, 1000)
Shape of y_val: (11405, 210)


In [43]:
#check these are differnt arrays!
np.array_equal(y_val, y_test)

False

### preparing the Embedding layer
compute an index mapping words ot known embeddings by parsing the data dump of pre-trained embeddings
NB stopwords haven't been removed yet...

In [44]:
embeddings_index = {}
#f = open(os.path.join(DATADIR, 'glove.6B.100d.txt'))
#for line in f:
##    values = line.split()
#    word = values[0]
#    coefs = np.asarray(values[1:], dtype='float32')
#    embeddings_index[word] = coefs
#f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 0 word vectors.


compute embedding matrix using embedding_index dict and word_index

In [49]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))# used 6B.100d.txt
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.

In [50]:
# embedding_layer = Embedding(len(word_index) + 1,
#                             EMBEDDING_DIM, # used 6B.100d.txt
#                             weights=[embedding_matrix],
#                             input_length= MAX_SEQUENCE_LENGTH, #MAX_SEQUENCE LENGTH
#                             trainable=False)

embedding_layer = Embedding(len(word_index) + 1, 
                            EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

### Estimate class weights for unbalanced datasets.
paramter to model.fit = __class_weight__: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

Implement class_weight from sklearn:

- Import the module 

`from sklearn.utils import class_weight`
- calculate the class weight, If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)):

`class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)`

- change it to a dict in order to work with Keras.

`class_weight_dict = dict(enumerate(class_weight))`

- Add to model fitting

`model.fit(X_train, y_train, class_weight=class_weight)`

In [26]:
# class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weight_dict = dict(enumerate(class_weight))

### Custom loss function

In [51]:
class WeightedBinaryCrossEntropy(object):

    def __init__(self, pos_ratio):
        neg_ratio = 1. - pos_ratio
        #self.pos_ratio = tf.constant(pos_ratio, tf.float32)
        self.pos_ratio = pos_ratio
        #self.weights = tf.constant(neg_ratio / pos_ratio, tf.float32)
        self.weights = neg_ratio / pos_ratio
        self.__name__ = "weighted_binary_crossentropy({0})".format(pos_ratio)

    def __call__(self, y_true, y_pred):
        return self.weighted_binary_crossentropy(y_true, y_pred)

    def weighted_binary_crossentropy(self, y_true, y_pred):
            # Transform to logits
            epsilon = tf.convert_to_tensor(K.common._EPSILON, y_pred.dtype.base_dtype)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
            y_pred = tf.log(y_pred / (1 - y_pred))

            cost = tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.weights)
            return K.mean(cost * self.pos_ratio, axis=-1)
    
y_true_arr = np.array([0,1,0,1], dtype="float32")
y_pred_arr = np.array([0,0,1,1], dtype="float32")
y_true = tf.constant(y_true_arr)
y_pred = tf.constant(y_pred_arr)

with tf.Session().as_default(): 
    print(WeightedBinaryCrossEntropy(0.5)(y_true, y_pred).eval())
    print(binary_crossentropy(y_true, y_pred).eval())


4.00756
8.01512


### difficulty getting global precision/recall metrics . CAUTION interpreting monitoring metrics
fcholltet: "Basically these are all global metrics that were approximated
batch-wise, which is more misleading than helpful. This was mentioned in
the docs but it's much cleaner to remove them altogether. It was a mistake
to merge them in the first place."

In [52]:
def mcor(y_true, y_pred):
     #matthews_correlation
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp * tn - fp * fn)
     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
     return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

## Training a 1D convnet

### 1. Create model

In [53]:
NB_CLASSES = y_train.shape[1]
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu', name = 'conv0')(embedded_sequences)

x = MaxPooling1D(5, name = 'max_pool0')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv1')(x)

x = MaxPooling1D(5 , name = 'max_pool1')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv2')(x)

x = MaxPooling1D(35, name = 'global_max_pool')(x)  # global max pooling

x = Flatten()(x) #reduce dimensions from 3 to 2; convert to vector + FULLYCONNECTED

x = Dense(128, activation='relu')(x)

x = Dense(NB_CLASSES, activation='sigmoid', name = 'fully_connected')(x)

# Create model. 
# This creates Keras model instance, will use this instance to train/test the model.
model = Model(sequence_input, x)

### 2. Compile model

In [54]:
model.compile(loss=WeightedBinaryCrossEntropy(POS_RATIO),
              optimizer='rmsprop',
              metrics=['binary_accuracy', precision, recall, f1])

Metric values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.

All metrics are reported in verbose output and in the history object returned from calling the fit() function. In both cases, the name of the metric function is used as the key for the metric values. In the case of metrics for the validation dataset, the “val_” prefix is added to the key.

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).


In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         21313300  
_________________________________________________________________
conv0 (Conv1D)               (None, 996, 128)          64128     
_________________________________________________________________
max_pool0 (MaxPooling1D)     (None, 199, 128)          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 195, 128)          82048     
_________________________________________________________________
max_pool1 (MaxPooling1D)     (None, 39, 128)           0         
_________________________________________________________________
conv2 (Conv1D)               (None, 35, 128)           82048     
__________

### Tensorboard callbacks /metrics /monitor training

<span style="color:red"> **Size of these files is killing storage during training. Is it histograms?**</span>

In [56]:
tb = TensorBoard(log_dir='/data/learn_embedding_logs', histogram_freq=0, write_graph=True, write_images=False)

In [ ]:
# class Metrics(Callback):
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []
 
#     def on_epoch_end(self, epoch, logs={}):
#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
#         _val_f1 = f1_score(val_targ, val_predict)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
#         self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
#         print("- val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
#         return
 
# metrics = Metrics()

### 3. Train model

In [58]:
# metrics callback causes: AttributeError: 'Model' object has no attribute 'validation_data'
# So disable for now

model.fit(
    x_train, y_train, 
    validation_data=(x_val, y_val), 
    epochs=10, batch_size=128, 
    callbacks=[tb]
)

Train on 91239 samples, validate on 11405 samples
Epoch 1/10
91239/91239 [==============================] - 468s 5ms/step - loss: 0.0112 - binary_accuracy: 0.9941 - precision: 0.7841 - recall: 0.2146 - f1: 0.3275 - val_loss: 0.0095 - val_binary_accuracy: 0.9947 - val_precision: 0.7616 - val_recall: 0.3604 - val_f1: 0.4883
Epoch 2/10
91239/91239 [==============================] - 468s 5ms/step - loss: 0.0080 - binary_accuracy: 0.9953 - precision: 0.8224 - recall: 0.4216 - f1: 0.5552 - val_loss: 0.0076 - val_binary_accuracy: 0.9955 - val_precision: 0.8163 - val_recall: 0.4693 - val_f1: 0.5951
Epoch 3/10
91239/91239 [==============================] - 468s 5ms/step - loss: 0.0066 - binary_accuracy: 0.9960 - precision: 0.8402 - recall: 0.5323 - f1: 0.6506 - val_loss: 0.0068 - val_binary_accuracy: 0.9960 - val_precision: 0.8281 - val_recall: 0.5439 - val_f1: 0.6558
Epoch 4/10
91239/91239 [==============================] - 472s 5ms/step - loss: 0.0057 - binary_accuracy: 0.9964 - precision: 0.

### Evaluate model

#### Training metrics

In [59]:
y_prob = model.predict(x_train)

In [60]:
y_prob.shape

(91239, 210)

In [61]:
y_pred = y_prob
y_pred[y_pred>=P_THRESHOLD] = 1
y_pred[y_pred<P_THRESHOLD] = 0

In [62]:
f1_score(y_train, y_pred, average='micro')

0.86933178553290291

In [63]:
#average= None, the scores for each class are returned.
precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([ 0.        ,  0.        ,  0.99511719,  0.80769231,  0.        ,
         0.77657267,  0.94117647,  0.9451146 ,  0.        ,  0.        ,
         0.        ,  0.90643275,  0.        ,  0.        ,  0.91496599,
         0.        ,  0.88034188,  0.        ,  0.        ,  0.86403807,
         0.8967686 ,  0.84624483,  0.95786865,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.98484848,  0.95588235,
         0.86842105,  1.        ,  0.        ,  0.75524476,  0.87901701,
         0.90116279,  0.82683215,  0.65116279,  0.83486239,  0.92738589,
         0.        ,  0.93762994,  0.        ,  0.        ,  0.76704545,
         0.        ,  0.        ,  0.89534884,  0.77895392,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.92210526,  0.33333333,  0.93115318,
         0.88888889,  0.        ,  0.79069767,  0.78099174,  0.        ,
         0.81013281,  0.        ,  1.        ,  0.8

In [64]:
a = precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
pd.DataFrame(list(a))

,0,1,2,3,4,5,6,7,8,9,...,200,201,202,203,204,205,206,207,208,209
0,0.0,0.0,0.995117,0.807692,0.0,0.776573,0.941176,0.945115,0.0,0.0,...,0.838926,0.808511,0.0,0.883030,0.0,0.909091,0.0,0.0,1.000000,0.963666
1,0.0,0.0,0.932296,0.695364,0.0,0.700587,0.505263,0.893896,0.0,0.0,...,0.737463,0.638655,0.0,0.924492,0.0,0.137931,0.0,0.0,0.382353,0.953838
2,0.0,0.0,0.962683,0.747331,0.0,0.736626,0.657534,0.918792,0.0,0.0,...,0.784929,0.713615,0.0,0.903286,0.0,0.239521,0.0,0.0,0.553191,0.958727
3,8.0,60.0,1093.000000,151.000000,12.0,511.000000,95.000000,1753.000000,4.0,21.0,...,339.000000,119.000000,6.0,1576.000000,8.0,145.000000,19.0,30.0,34.000000,1863.000000


In [66]:
f1_byclass = pd.DataFrame((a)[2], columns=['f1'])


In [67]:
support_byclass = pd.DataFrame((a)[3], columns=['support'])

f1_byclass = pd.merge(
    left=f1_byclass, 
    right=support_byclass, 
    left_index=True,
    right_index=True,
    how='outer', 
    validate='one_to_one'
)

f1_byclass['index_col'] = f1_byclass.index

In [68]:
f1_byclass['level2taxon'] = f1_byclass['index_col'].map(labels_index).copy()

In [69]:
print("At p_threshold of {}, there were {} out of {} ({})% taxons with auto-tagged content"
      .format(P_THRESHOLD, 
              f1_byclass.loc[f1_byclass['f1'] > 0].shape[0], 
              y_pred.shape[1], 
              (f1_byclass.loc[f1_byclass['f1'] > 0].shape[0]/y_pred.shape[1])*100 ))

At p_threshold of 0.4, there were 121 out of 210 (57.61904761904761)% taxons with auto-tagged content


In [70]:
no_auto_content = f1_byclass.loc[f1_byclass['f1'] == 0]
no_auto_content = no_auto_content.set_index('level2taxon')

In [71]:
no_auto_content['support'].sort_values().plot( kind = 'barh', figsize=(20, 20))

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [72]:
classes_predictedto = f1_byclass.loc[f1_byclass['f1'] > 0]
classes_predictedto = classes_predictedto.set_index('level2taxon') 

In [73]:
classes_predictedto.plot.scatter(x='support', y='f1', figsize=(10, 20), xticks=np.arange(0, 9700, 100))

In [74]:
classes_predictedto['f1'].sort_values().plot( kind = 'barh', figsize=(20, 20))

In [75]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_train, y_pred, average='micro', sample_weight=None) 

(0.89209777659349354, 0.84769883824843606, 0.86933178553290291, None)

In [76]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_train, y_pred, average='macro', sample_weight=None)

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.51256145742465209, 0.39793437745990079, 0.42912917918654758, None)

#### Validation metrics

In [77]:
y_pred_val = model.predict(x_val)

In [78]:
y_pred_val[y_pred_val>=P_THRESHOLD] = 1
y_pred_val[y_pred_val<P_THRESHOLD] = 0

In [79]:
#average= None, the scores for each class are returned.
precision_recall_fscore_support(y_val, y_pred_val, average=None, sample_weight=None)

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(array([ 0.        ,  0.        ,  0.94776119,  0.4       ,  0.        ,
         0.50943396,  0.        ,  0.72767857,  0.        ,  0.        ,
         0.        ,  0.72      ,  0.        ,  0.        ,  0.68      ,
         0.        ,  0.4       ,  0.        ,  0.        ,  0.77058824,
         0.74295474,  0.60309278,  0.84951456,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.83333333,  0.77777778,
         0.90909091,  0.        ,  0.        ,  0.55172414,  0.78402904,
         0.63157895,  0.61627907,  0.25      ,  0.55555556,  0.66666667,
         0.        ,  0.71698113,  0.        ,  0.        ,  0.35      ,
         0.        ,  0.        ,  0.73684211,  0.58706468,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.74137931,  0.        ,  0.78151261,
         0.        ,  0.        ,  1.        ,  0.38888889,  0.        ,
         0.50974026,  0.        ,  0.        ,  0.5

In [80]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_val, y_pred_val, average='micro', sample_weight=None) 

(0.73569706964353676, 0.68261534839924676, 0.70816289150741807, None)

In [81]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_val, y_pred_val, average='macro', sample_weight=None)

/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/envs/govuk_taxonomy/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.37119671647455299, 0.28011837432387876, 0.30703269497579072, None)

## Tag unlabelled content

In [82]:
def get_predictions(df, level1taxon=False):
    #process data for model input
    texts = df['combined_text']
    tokenizer = Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts) #yield one sequence per input text

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    x_new = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH
    
    print('Shape of untagged tensor:', x_new.shape)
    
    #predict tag for untagged data
    y_pred_new = model.predict(x_new)
    
    #get model output into pandas & get a column to track index for later merge
    y_pred_new = pd.DataFrame(y_pred_new)
    y_pred_new['index_col'] = y_pred_new.index
    
    #Make long by taxon so easier to filter rows and examine effect of p_threshold
    y_pred_new = pd.melt(y_pred_new, id_vars=['index_col'],
                             var_name='level2taxon_code', value_name='probability')
    
    #get taxon names
    y_pred_new['level2taxon'] = y_pred_new['level2taxon_code'].map(labels_index)
    
    if level1taxon==False:
        #get the info about the content
        new_info = df[[ 'base_path', 'content_id', 'title', 'description', 
                   'document_type', 'publishing_app', 'locale']]
    else:
        new_info = df[[ 'base_path', 'content_id', 'title', 'description', 
                   'document_type', 'publishing_app', 'locale', 'level1taxon']]
    
    
    #merge content info with taxon prediction
    pred_new = pd.merge(
    left=new_info, 
    right=y_pred_new, 
    left_index=True,
    right_on='index_col',
    how='outer'
    )
    
    #drop the cols needed for mergingin and naming
    pred_new.drop(['level2taxon_code', 'index_col'], axis=1, inplace = True)
    
    return pred_new
    

### Labelled at level1only

In [84]:
#read in untagged content
untagged_raw = pd.read_csv(os.path.join(DATADIR, 'untagged_content.csv.gz'), dtype=object, compression='gzip')

In [85]:
#make a copy so you can edit data without needed to read in each time
untagged_df = untagged_raw.copy()

In [86]:
pred_untagged = get_predictions(untagged_df )

Found 157451 unique tokens.
Shape of untagged tensor: (57337, 1000)


In [87]:
#keep only rows where prob of taxon > 0.5
pred_untagged = pred_untagged.loc[pred_untagged['probability'] > 0.5]

In [88]:
#write to csv
pred_untagged.to_csv(os.path.join(DATADIR, 'predictions_for_untagged.csv.gz'), index=False, compression='gzip')

### Untagged (includes old tags)

In [89]:
labelled_level1 = pd.read_csv(os.path.join(DATADIR, 'labelled_level1.csv.gz'), dtype=object, compression='gzip')

In [90]:
pred_labelled_level1 = get_predictions(labelled_level1)

Found 101038 unique tokens.
Shape of untagged tensor: (32358, 1000)


In [91]:
#keep only rows where prob of taxon > 0.5
pred_labelled_level1 = pred_labelled_level1.loc[pred_labelled_level1['probability'] > 0.5]

In [92]:
#write to csv
pred_labelled_level1.to_csv(os.path.join(DATADIR, 'predictions_for_level1only.csv.gz'), index=False, compression='gzip')